In [79]:
from model import DCNN
import librosa
import librosa.display as ldp
import torch
import os
import torchvision.transforms as transforms
import numpy as np

In [80]:
model = DCNN(0.25)

In [81]:
# Test the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
load_model = DCNN(0.25).to(device)
load_model.load_state_dict(torch.load('../models/voice-classification.pth', map_location=torch.device(device)))

<All keys matched successfully>

# Record audio

In [100]:
import pyaudio
import wave
import time
 
# set the chunk size and sample rate
chunk = 1024
sample_format = pyaudio.paInt16
channels = 1
fs = 44100  # sample rate
 
# create an instance of the PyAudio class
p = pyaudio.PyAudio()

# open a stream to record audio
stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

# record for 5 seconds
print("Speak")
for i in reversed(range(3)):
    print(i)
    time.sleep(1)
seconds = 2
frames = []
for i in range(int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)
print("End")
 
# stop the stream and close the PyAudio instance
stream.stop_stream()
stream.close()
p.terminate()
 
# save the recorded audio as a WAV file
wf = wave.open("../data/output.wav", "wb")
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b"".join(frames))
wf.close()


Speak
2
1
0
End


# Read Audio File


In [108]:
def preprocess(audio_file):
    # load the audio in wave form
    waveform_audio, sample_rate = librosa.load(audio_file)
    
    # Convert image to spectogram short time fourier transform
    waveform_audio_stft = librosa.stft(waveform_audio)
    
    # Convert image to amplitude_db form
    s_db = librosa.amplitude_to_db(np.abs(waveform_audio_stft), ref=np.max )
    
    # Convert to tensor
    s_db = torch.tensor(s_db).unsqueeze(0)
    
    # Resize 
    transform = transforms.Resize((1025,44), antialias=False)
    s_db = transform(s_db)
    
    # Reuturn image form of audio
    return s_db

In [114]:
audio_file = '../data/012c8314_nohash_0_dog.wav'

In [110]:
audio = preprocess(audio_file)

In [111]:
index = torch.argmax(torch.softmax(model(audio.unsqueeze(dim=0).to(device)), dim=1)).item()

In [112]:
labels=['dog', 'bird', 'cat']

In [113]:
print(index,labels[index])

0 dog
